## Google Collab

In [4]:
!lscpu |grep 'Model name'
!lscpu |grep 'Core(s) per socket:'
!free -h
!lscpu |grep 'Thread(s) per core'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Core(s) per socket:  2
              total        used        free      shared  buff/cache   available
Mem:            25G        868M         22G        1.2M        2.1G         24G
Swap:            0B          0B          0B
Thread(s) per core:  2


In [3]:
# Mount Google Drive and load project 2
# WARNING, we have to add the shared drive of Sami on our home directory (by creating an alias)
from google.colab import drive
drive.mount('/content/drive')
#Move to the shared directory
%cd /content/drive/MyDrive/ml_project_2_drive/ml_project_2/
# list all files
! ls

# Read helpers python file
!cp /content/drive/MyDrive/ml_project_2_drive/ml_project_2/script/helper_functions.py .

# A good help can be found here:
#https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ml_project_2_drive/ml_project_2
data		     project2_description.pdf  road_segmentation_sami.ipynb
helper_functions.py  README.md		       script


In [ ]:
!git config --global user.email "sami.ferchiou@epfl.ch"
!git config --global user.name "samiferchiou"

In [ ]:
!git pull

In [ ]:
!git add .
!git commit -m "update from google colab"
!git push

## library

In [5]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def load_train_dataset():
    #root_dir = "../data/training/"
    root_dir = "/content/drive/MyDrive/ml_project_2_drive/ml_project_2/data/training/"
    image_dir = root_dir + "images/"
    gt_dir = root_dir + "groundtruth/"
    files = os.listdir(image_dir)
    n = len(files)

    np_X = [load_image(image_dir + files[i]) for i in range(n)]
    np_Y = [load_image(gt_dir + files[i]) for i in range(n)]

    torch_X = [torch.from_numpy(x).reshape(3, 400, 400) for x in np_X]
    torch_Y =[torch.from_numpy(y).reshape(1, 400, 400) for y in np_Y]

    return (torch_X, torch_Y)

In [6]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state:            checkpoint we want to save
    is_best:          boolean to indicates if it is the best checkpoint
    checkpoint_path:  path to save checkpoint
    best_model_path:  path to save best model
    """
    torch.save(state, checkpoint_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(checkpoint_path, best_fpath)

## Imports

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import random
import os,sys
from PIL import Image
import torchvision.transforms as T
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#from helper_functions import *

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Constants definition

In [9]:
CHECKPOINT_PATH ="/content/drive/MyDrive/ml_project_2_drive/ml_project_2/checkpoint/current_checkpoint.pt"
BEST_MODEL_PATH ="/content/drive/MyDrive/ml_project_2_drive/ml_project_2/checkpoint/best_model.pt"
NBR_EPOCHS = 5
BATCH_SIZE = 1
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0
GAMMA = 1
K_FOLD = 4
VALIDATION_SET_IDX = 0
BATCH_SIZE_VAL = 5
SEED = 0

## Data set

In [10]:
class imagesDataset(Dataset): 
    def __init__(self, K_fold, validation_set_idx, batch_size_val, seed):
        X, Y = load_train_dataset()

        #tresholding ground_truth values
        Y = [(y > 0.5).long() for y in Y]
        shape_y = Y[0].shape
        
        #shuffling
        random.seed(seed)
        idx_list = list(range(len(X)))
        random.shuffle(idx_list)
        random.seed()
        X = [X[idx] for idx in idx_list]
        Y = [Y[idx] for idx in idx_list]
        
        #K_fold separation
        fold_size = len(X) // K_fold
        start_validation_idx = validation_set_idx * fold_size
        end_validation_idx = start_validation_idx + fold_size
        self.X_train = X[slice(0, start_validation_idx)] + X[slice(end_validation_idx, None)]
        self.Y_train = Y[slice(0, start_validation_idx)] + Y[slice(end_validation_idx, None)]
        self.X_validation = X[slice(start_validation_idx, end_validation_idx)]
        self.Y_validation = Y[slice(start_validation_idx, end_validation_idx)]

        #data augmentation
        #self.X_train = compose_all_functions_for_data(self.X_train)
        #self.Y_train = compose_all_functions_for_data(self.Y_train)
        #self.X_validation = compose_all_functions_for_data(self.X_validation)
        #self.Y_validation = compose_all_functions_for_data(self.Y_validation)
        self.n_samples = len(self.X_train)
        
        #casting into tensors
        self.X_train = torch.stack(self.X_train)
        self.X_validation = torch.stack(self.X_validation)
        self.Y_train = torch.reshape(torch.stack(self.Y_train) , (-1, shape_y[1], shape_y[2]))
        self.Y_validation = torch.reshape(torch.stack(self.Y_validation) , (-1, shape_y[1], shape_y[2]))

        #creating dataloader for validation data
        class dataset_validation(Dataset):
            def __init__(s,x,y):
                s.x = x
                s.y = y
                s.size = len(s.x)
            def __getitem__(s, index):
                return s.x[index], s.y[index]
            def __len__(s):
                return s.size
               
        self.validation_data_loader = torch.utils.data.DataLoader(
            dataset_validation(self.X_validation, self.Y_validation),
            batch_size = batch_size_val, shuffle = False)
        
        
    def __getitem__(self, index):
        return self.X_train[index], self.Y_train[index]

    def __len__(self):
        return self.n_samples
    
    def get_validation_dataloader(self):
        return self.validation_data_loader


dataset = imagesDataset(K_FOLD, VALIDATION_SET_IDX, BATCH_SIZE_VAL, SEED)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True)
validation_dataloader = dataset.get_validation_dataloader()

## Model

In [11]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # 
        self.pool_d = nn.MaxPool2d(2, 2)
        self.pool_u = nn.Upsample(scale_factor=2)
        
        # Activation function
        self.activ = nn.ReLU()
        #self.activ = nn.LeakyReLU(0.1)
        
        # Convolution Downwards
        self.conv_1 = nn.Conv2d(3, 64, (3,3), padding=(1, 1))
        self.conv_2 = nn.Conv2d(64, 64, (3,3), padding=(1, 1))
        
        self.conv_3 = nn.Conv2d(64, 128, (3,3), padding=(1, 1))
        self.conv_4 = nn.Conv2d(128, 128, (3,3), padding=(1, 1))
        
        self.conv_5 = nn.Conv2d(128, 256, (3,3), padding=(1, 1))
        self.conv_6 = nn.Conv2d(256, 256, (3,3), padding=(1, 1))
        
        self.conv_7 = nn.Conv2d(256, 512, (3,3), padding=(1, 1))
        self.conv_8 = nn.Conv2d(512, 512, (3,3), padding=(1, 1))
        
        self.conv_9 = nn.Conv2d(512, 1024, (3,3), padding=(1, 1))
        self.conv_10 = nn.Conv2d(1024, 1024, (3,3), padding=(1, 1))
        
        
        # Upconvolution
        self.upconv_1 = nn.Conv2d(512+1024, 512, (3,3), padding=(1, 1))
        self.upconv_2 = nn.Conv2d(512, 512, (3,3), padding=(1, 1))
        
        self.upconv_3 = nn.Conv2d(256+512, 256, (3,3), padding=(1, 1))
        self.upconv_4 = nn.Conv2d(256, 256, (3,3), padding=(1, 1))
        
        self.upconv_5 = nn.Conv2d(128+256, 128, (3,3), padding=(1, 1))
        self.upconv_6 = nn.Conv2d(128, 128, (3,3), padding=(1, 1))
        
        self.upconv_7 = nn.Conv2d(64+128, 64, (3,3), padding=(1, 1))
        self.upconv_8 = nn.Conv2d(64, 64, (3,3), padding=(1, 1))
        self.upconv_9 = nn.Conv2d(64, 2, (1,1))


    def forward(self, x):
        # Convolution with activation and max_pooling
        xd_1 = self.activ(self.conv_1(x))
        xd_2 = self.activ(self.conv_2(xd_1))
    
        xd_3 = self.activ(self.conv_3(self.pool_d(xd_2)))
        xd_4 = self.activ(self.conv_4(xd_3))
        
        xd_5 = self.activ(self.conv_5(self.pool_d(xd_4)))
        xd_6 = self.activ(self.conv_6(xd_5))
        
        xd_7 = self.activ(self.conv_7(self.pool_d(xd_6)))
        xd_8 = self.activ(self.conv_8(xd_7))
        
        xd_9 = self.activ(self.conv_9(self.pool_d(xd_8)))
        xd_10 = self.pool_u(self.activ(self.conv_10(xd_9)))

        # "Fractionally / Backward strided convolution" with activation and upsampling
        xu_1 = self.activ(self.upconv_1(torch.cat((xd_8, xd_10), dim=1)))
        xu_2 = self.pool_u(self.activ(self.upconv_2(xu_1)))
        
        xu_3 = self.activ(self.upconv_3(torch.cat((xd_6, xu_2), dim=1)))
        xu_4 = self.pool_u(self.activ(self.upconv_4(xu_3)))
        
        xu_5 = self.activ(self.upconv_5(torch.cat((xd_4, xu_4), dim=1)))
        xu_6 = self.pool_u(self.activ(self.upconv_6(xu_5)))
        
        xu_7 = self.activ(self.upconv_7(torch.cat((xd_2, xu_6), dim=1)))
        xu_8 = self.activ(self.upconv_8(xu_7))
        xu_9 = self.upconv_9(xu_8)
    
        return xu_9

###################################################################################################################################
###################################################################################################################################

class ConvNet_not_recurrent(nn.Module):
    def __init__(self):
        super(ConvNet_not_recurrent, self).__init__()
        self.pool_d = nn.MaxPool2d(2, 2)
        self.pool_u = nn.Upsample(scale_factor=2)
        
        # Activation function
        self.activ = nn.ReLU()
        #self.activ = nn.LeakyReLU(0.1)
        
        # Convolution Downwards
        self.conv_1 = nn.Conv2d(3, 64, (3,3), padding=(1, 1))
        self.conv_2 = nn.Conv2d(64, 64, (3,3), padding=(1, 1))
        
        self.conv_3 = nn.Conv2d(64, 128, (3,3), padding=(1, 1))
        self.conv_4 = nn.Conv2d(128, 128, (3,3), padding=(1, 1))
        
        self.conv_5 = nn.Conv2d(128, 256, (3,3), padding=(1, 1))
        self.conv_6 = nn.Conv2d(256, 256, (3,3), padding=(1, 1))
        
        self.conv_7 = nn.Conv2d(256, 512, (3,3), padding=(1, 1))
        self.conv_8 = nn.Conv2d(512, 512, (3,3), padding=(1, 1))
        
        self.conv_9 = nn.Conv2d(512, 1024, (3,3), padding=(1, 1))
        self.conv_10 = nn.Conv2d(1024, 1024, (3,3), padding=(1, 1))
        
        
        # Upconvolution
        self.upconv_1 = nn.Conv2d(1024, 512, (3,3), padding=(1, 1))
        self.upconv_2 = nn.Conv2d(512, 512, (3,3), padding=(1, 1))
        
        self.upconv_3 = nn.Conv2d(512, 256, (3,3), padding=(1, 1))
        self.upconv_4 = nn.Conv2d(256, 256, (3,3), padding=(1, 1))
        
        self.upconv_5 = nn.Conv2d(256, 128, (3,3), padding=(1, 1))
        self.upconv_6 = nn.Conv2d(128, 128, (3,3), padding=(1, 1))
        
        self.upconv_7 = nn.Conv2d(128, 64, (3,3), padding=(1, 1))
        self.upconv_8 = nn.Conv2d(64, 64, (3,3), padding=(1, 1))
        self.upconv_9 = nn.Conv2d(64, 2, (1,1))



    def forward(self, x):
        # Convolution with activation and max_pooling
        xd_1 = self.activ(self.conv_1(x))
        xd_2 = self.activ(self.conv_2(xd_1))
    
        xd_3 = self.activ(self.conv_3(self.pool_d(xd_2)))
        xd_4 = self.activ(self.conv_4(xd_3))
        
        xd_5 = self.activ(self.conv_5(self.pool_d(xd_4)))
        xd_6 = self.activ(self.conv_6(xd_5))
        
        xd_7 = self.activ(self.conv_7(self.pool_d(xd_6)))
        xd_8 = self.activ(self.conv_8(xd_7))
        
        xd_9 = self.activ(self.conv_9(self.pool_d(xd_8)))
        xd_10 = self.pool_u(self.activ(self.conv_10(xd_9)))

        # "Fractionally / Backward strided convolution" with activation and upsampling
        xu_1 = self.activ(self.upconv_1(xd_10))
        xu_2 = self.pool_u(self.activ(self.upconv_2(xu_1)))
        
        xu_3 = self.activ(self.upconv_3(xu_2))
        xu_4 = self.pool_u(self.activ(self.upconv_4(xu_3)))
        
        xu_5 = self.activ(self.upconv_5(xu_4))
        xu_6 = self.pool_u(self.activ(self.upconv_6(xu_5)))
        
        xu_7 = self.activ(self.upconv_7(xu_6))
        xu_8 = self.activ(self.upconv_8(xu_7))
        xu_9 = self.upconv_9(xu_8)
        return xu_9


## Training

In [12]:
def train(n_epochs, data_loader, model, optimizer, scheduler, criterion, device, checkpoint_path, best_model_path):
    train_loader = data_loader
    validation_loader = train_loader.dataset.get_validation_dataloader()
    f1_max = 0
    
    for epoch in range(n_epochs):
        loss_list = []
        model.train()
        for (data, target) in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        loss_epoch = np.mean(loss_list)

        model.eval()
        #computing F1 score on validation data
        tp, fp, fn = 0, 0, 0
        for (data, target) in validation_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            prediction = torch.argmax(output, dim = 1)
            confusions = prediction / target
            tp += torch.sum(confusions == 1).item()
            print("tp : ", tp)
            fp += torch.sum(confusions == float('inf')).item()
            print("fp : ", fp)
            fn += torch.sum(confusions == 0).item()
            print("fn : ", fn)            
        f1_score_val = 2 * tp / (2 * tp + fp + fn)
        
        if f1_score_val > f1_max:
            torch.save(model.state_dict(), best_model_path)
            f1_max = f1_score_val

        checkpoint = {
            'epoch': epoch,
            'f1_max': f1_max,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.get_last_lr(),
        }
        
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        #computing F1 score on training data
        tp, fp, fn = 0, 0, 0
        for (data, target) in train_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            prediction = torch.argmax(output, dim = 1)
            confusions = prediction / target
            tp += torch.sum(confusions == 1).item()
            fp += torch.sum(confusions == float('inf')).item()
            fn += torch.sum(confusions == 0).item()
        f1_score_train = 2 * tp / (2 * tp + fp + fn)

        print(f"Epoch {epoch} || Loss:{loss_epoch:.6f} || Training F1 {f1_score_train:.6f}|| Validation F1 {f1_score_val:.6f} || Learning rate {scheduler.get_last_lr()[0]:.6f}"+"\n")  
        scheduler.step()
        
    return 

In [14]:
LOAD_LAST_CHECKPOINT = False
LOAD_BEST_MODEL = False

#model = ConvNet().to(device)
model = ConvNet_not_recurrent().to(device)

if LOAD_LAST_CHECKPOINT and not LOAD_BEST_MODEL:
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['state_dict'])
    
if not LOAD_LAST_CHECKPOINT and LOAD_BEST_MODEL:
    model.load_state_dict(torch.load(BEST_MODEL_PATH))

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ExponentialLR(optimizer, GAMMA)
criterion = nn.CrossEntropyLoss()

In [15]:
train(NBR_EPOCHS, train_loader, model, optimizer, scheduler, criterion, device, CHECKPOINT_PATH, BEST_MODEL_PATH)

tp :  0
fp :  0
fn :  147493
tp :  0
fp :  0
fn :  299768
tp :  0
fp :  0
fn :  461833
tp :  0
fp :  0
fn :  633557
tp :  0
fp :  0
fn :  793231
Epoch 0 || Loss:0.706831 || Training F1 0.000000|| Validation F1 0.000000 || Learning rate 0.001000

tp :  0
fp :  0
fn :  147493
tp :  0
fp :  0
fn :  299768
tp :  0
fp :  0
fn :  461833
tp :  0
fp :  0
fn :  633557
tp :  0
fp :  0
fn :  793231
Epoch 1 || Loss:0.535013 || Training F1 0.000000|| Validation F1 0.000000 || Learning rate 0.001000

tp :  0
fp :  0
fn :  147493
tp :  0
fp :  0
fn :  299768
tp :  0
fp :  0
fn :  461833
tp :  0
fp :  0
fn :  633557
tp :  0
fp :  0
fn :  793231
Epoch 2 || Loss:0.513717 || Training F1 0.000000|| Validation F1 0.000000 || Learning rate 0.001000

tp :  0
fp :  0
fn :  147493
tp :  0
fp :  0
fn :  299768
tp :  0
fp :  0
fn :  461833
tp :  0
fp :  0
fn :  633557
tp :  0
fp :  0
fn :  793231
Epoch 3 || Loss:0.508713 || Training F1 0.000000|| Validation F1 0.000000 || Learning rate 0.001000

tp :  0
fp :  0
